In [74]:
#@title Banner
HTML('''
<div style="position: relative;">
    <div style="
        background-image: url(
          'https://images.unsplash.com/photo-1631653355789-3e32dcce22db?q=80&w=1964&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D'
          );
        background-size: cover;
        background-position: center;
        width: 100%;
        height: 300px;
        border-radius: 10px;
        margin: 20px 0;
        box-shadow: 0 4px 8px rgba(0,0,0,0.1);">
    </div>
    <div style="
        position: absolute;
        bottom: 20px;
        left: 20px;
        color: white;
        font-size: 24px;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.5);">
        Lung Ventilator System
    </div>
</div>
''')

Photo by [The Centers for Disease Control and Prevention](https://unsplash.com/@cdc) on Unsplash.

# **Ventilator Pressure Prediction 💨**

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla blandit dictum quam sit amet elementum. Vestibulum sed augue eu augue euismod ornare at lobortis nibh. Aenean hendrerit non dui a faucibus. In ut tincidunt orci. Nunc quis scelerisque diam. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. Integer finibus lobortis nisi, venenatis sagittis ipsum cursus ac. Suspendisse nunc libero, bibendum a condimentum non, rhoncus vitae mauris. Vestibulum ullamcorper ligula tempor dui bibendum tempor. Donec blandit egestas lectus. Praesent finibus justo at dictum tempus. Nullam elementum sodales libero, semper tempus nisl pulvinar eu. Nunc eget ipsum arcu. Donec ullamcorper libero vel eleifend condimentum. Vivamus sit amet tincidunt mi. Integer malesuada augue a purus porttitor, ut semper odio pretium.

**Authors:**
- [Daniel Valdez](https://github.com/Danval-003)
- [Emilio Solano](https://github.com/emiliosolanoo21)
- [Adrian Flores](https://github.com/adrianRFlores)
- [Andrea Ramírez](https://github.com/Andrea-gt)

***

## **(i) Problem Statement and Analysis**

La ventilación mecánica se ha convertido en un procedimiento médico esencial que implica el uso de ventiladores para administrar oxígeno a pacientes sedados  a través de un tubo endotraqueal. Sin embargo, este proceso enfrenta importantes desafíos, como se evidenció durante los primeros días de la pandemia de COVID-19, principalmente debido a la necesidad de supervisión clínica intensiva. Además, el desarrollo de nuevos métodos para controlar estos ventiladores resulta prohibitivamente costoso, incluso antes de llegar a la fase de ensayos clínicos.

En respuesta a estos retos, Google Brain, en colaboración con la Universidad de Princeton, está liderando una iniciativa para revolucionar el campo de la ventilación mecánica mediante la aplicación de machine learning. Su enfoque busca superar las limitaciones de los simuladores actuales, que funcionan como un conjunto donde cada modelo simula una única configuración pulmonar. La propuesta  considera que los pulmones y sus atributos forman un espacio continuo, por lo que es necesario explorar un enfoque paramétrico que tenga en cuenta las diferencias entre los pulmones de los pacientes.

El objetivo principal de este proyecto es desarrollar sistemas más sofisticados que puedan generalizar mejor entre pulmones con características variables, superando las capacidades de los controladores PID que son el estándar actual en la industria. Este avance podría reducir significativamente las barreras económicas en el desarrollo de nuevos métodos de control de ventiladores mecánicos, facilitando la creación de algoritmos que se adapten a las necesidades específicas de cada paciente. En última instancia, esto podría resultar en un acceso más amplio a tratamientos de ventilación y una reducción significativa de la carga de trabajo para el personal clínico, tanto en situaciones de crisis como en la práctica médica cotidiana.

## **(1) Import Libraries** ⬇️

In [52]:
# ===== Standard Libraries =====
import os  # For operating system utilities
import warnings  # For warning management

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# ===== Data Manipulation and Analysis =====
import numpy as np  # For numerical computations
import pandas as pd  # For data manipulation and analysis

# ===== Visualization =====
import matplotlib.pyplot as plt  # For basic plotting
import seaborn as sns  # For enhanced data visualization
from IPython.display import Image  # For displaying images in Jupyter Notebooks
from IPython.core.display import HTML  # For displaying HTML content
import base64  # For encoding/decoding binary data

# Set default figure size and style
plt.rcParams['figure.figsize'] = (6, 4)  # Default figure size
sns.set(style="whitegrid")  # Seaborn style for plots

# Define custom color palette
palette = sns.color_palette("viridis", 12)

# Enable inline plotting for Jupyter Notebooks
%matplotlib inline

# ===== File Handling =====
from google.colab import files  # For file upload and download in Colab

# ===== Progress Tracking =====
from tqdm import tqdm  # For progress bars in loops

# ===== Machine Learning and Deep Learning =====
import tensorflow as tf  # For deep learning models
from tensorflow import keras  # High-level neural networks API

# Set a fixed seed for reproducibility
random_state = 42
np.random.seed(random_state)  # Seed for NumPy random functions
tf.random.set_seed(random_state)  # Seed for TensorFlow random functions

In [5]:
# %pip install kaggle

In [8]:
# !pip install keras-tuner

## **(2) Data Upload** 📄

### **(1) Downloading Datasets from Kaggle**

In [14]:
# Upload files from the local system to the Colab environment
uploaded = files.upload()

# Iterate over the files uploaded by the user
for fn in uploaded.keys():
    # Print information about each uploaded file, including its name and size in bytes
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

# Create the directory ~/.kaggle if it does not exist. This directory is where Kaggle API expects configuration files.
!mkdir -p ~/.kaggle/

# Move the uploaded kaggle.json file into the ~/.kaggle/ directory. This file contains API credentials for Kaggle.
!mv kaggle.json ~/.kaggle/

# Set the permissions of kaggle.json to be readable only by the owner. This is necessary for security reasons.
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 68 bytes


In [16]:
 # !kaggle competitions download -c ventilator-pressure-prediction

In [18]:
 # !unzip /content/ventilator-pressure-prediction.zip

### **(2) Loading the Training Dataset**

In [20]:
# Load the training dataset into a DataFrame
df = pd.read_csv('train.csv')  # Read the CSV file 'train.csv' as a DataFrame

# Display the first five rows of the DataFrame
df.head()  # Preview the top entries to understand the data structure

,id,breath_id,R,C,time_step,u_in,u_out,pressure
0,1,1,20,50,0.000000,0.083334,0,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794
2,3,1,20,50,0.067514,22.509278,0,7.876254
3,4,1,20,50,0.101542,22.808822,0,11.742872
4,5,1,20,50,0.135756,25.355850,0,12.234987


## **(3) Exploratory Data Analysis 🔎**

Los datos empleados en la competencia se obtuvieron mediante un ventilador de código abierto modificado, conectado a un pulmón artificial de fuelle a través de un circuito respiratorio. El sistema incorpora dos variables de control fundamentales: la primera es una variable continua que oscila entre 0 y 100, representando el porcentaje de apertura de la válvula solenoide inspiratoria que permite la entrada de aire al pulmón (donde **0 indica un cierre completo** sin entrada de aire, y **100 una apertura total**). La segunda variable es binaria y controla la válvula exploratoria, indicando si está **abierta (1) o cerrada (0)** para permitir la salida del aire.

En este contexto, se reciben múltiples series temporales de respiraciones y se debe de desarrollar modelos para predecir la presión de la vía aérea en el circuito respiratorio durante la respiración, basándose en las series temporales de las variables de control mencionadas antes.

### **(1) Dataset Overview and Description**

In [22]:
# Print the number of records in the DataFrame
print("The given dataset has {:,} records and {:,} columns.".format(df.shape[0], df.shape[1]))

The given dataset has 6,036,000 records and 8 columns.


**Observaciones 💡 -->**
> - El conjunto de datos `train.csv`, con el que se está trabajando cuenta con 6,036,000 registros y 8 columnas, lo que indica que se trata de un conjunto de datos con una dimensión relativamente grande y  moderadamente detallado.

> - Cada serie temporal representa una respiración de aproximadamente 3 segundos. Cada fila es un paso de tiempo en una respiración y proporciona las dos señales de control, la presión resultante en las vías respiratorias y los atributos relevantes del pulmón.

> - Nótese que cada ciclo de respiración (`breath_id`) se registra en múltiples intervalos de tiempo, proporcionando mediciones continuas de `u_in`, `u_out` y `pressure`.

**Fuente:** [Página oficial de Kaggle](https://www.kaggle.com/competitions/ventilator-pressure-prediction/overview)

In [24]:
# Basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   breath_id  int64  
 2   R          int64  
 3   C          int64  
 4   time_step  float64
 5   u_in       float64
 6   u_out      int64  
 7   pressure   float64
dtypes: float64(3), int64(5)
memory usage: 368.4 MB


**Observaciones 💡 -->**

Como se puede observar en la pequeña tabla colocada anteriormente, se cuenta con las siguietes columnas en el conjunto de datos:
Aquí tienes una lista explicando las columnas del conjunto de datos de manera sencilla:

> - **id**: Identificador único para cada paso de tiempo en todo el archivo.

> - **breath_id**: Identificador único para cada ciclo de respiración.

> - **R**: Este valor indica cuán restringido está el paso del aire por las vías respiratorias, medido en cmH2O por litro por segundo. Es como la resistencia que sientes al tratar de inflar un globo a través de una pajilla: una pajilla más estrecha (alto R) hará que sea más difícil inflar el globo, mientras que una más ancha (bajo R) lo hará más fácil.

> - **C**: Este valor indica cuán flexible es el pulmón, medido en mililitros por cmH2O. Representa el cambio en el volumen de aire por cada cambio en la presión. Siguiendo con el ejemplo del globo, un globo más delgado (alto C) se inflará más fácilmente que uno más grueso (bajo C).

> - **time_step**: Este es el instante exacto en el que se realizó cada medición.

> - **u_in**: Este es el control para una válvula que permite la entrada de aire (inspiración) al pulmón. Sus valores varían entre 0 y 100, donde 0 significa que no se está permitiendo la entrada de aire y 100 significa que se está permitiendo la entrada máxima.

> - **u_out**: Este es el control para una válvula que permite la salida de aire (espiración) del pulmón. Puede tener un valor de 0 (sin salida de aire) o 1 (salida de aire permitida).

> - **pressure**: Este valor mide la presión en las vías respiratorias del circuito respiratorio, expresado en cmH2O. Indica cuánta presión se está ejerciendo en el aire dentro de las vías respiratorias durante la respiración.

### **(2) Variable Classification and Description**

| **Nombre**       | **Descripción**                         | **Tipo de Variable**         |
|------------------|-----------------------------------------|------------------------------|
| id               | Identificador único                     | Cualitativa (Nominal)        |
| breath_id        | Identificador de ciclo de respiración   | Cualitativa (Nominal)        |
| R                | Resistencia de las vías respiratorias   | Cuantitativa (Continua)      |
| C                | Cumplimiento del pulmón                 | Cuantitativa (Continua)      |
| time_step        | Marca de tiempo de cada medición        | Cuantitativa (Continua)      |
| u_in             | Entrada de aire en la válvula           | Cuantitativa (Discreta)      |
| u_out            | Salida de aire en la válvula            | Cualitativa (Nominal)        |
| pressure         | Presión en las vías respiratorias       | Cuantitativa (Continua)      |


**Observaciones 💡 -->**
> - Se observa que el conjunto de datos incluye tres variables cualitativas, todas de tipo nominal, y cinco variables cuantitativas que se distribuyen entre continuas y discretas.

### **(3) Preliminary Data Exploration and Cleaning**

#### **(1) Data Preprocessing and Preparation**